# Embeddings Work
##### By Pranjal Chatterjee
Exploratory notebook for working on birdnet embeddings

### Imports and Setup

In [1]:
import pandas as pd
import os

In [2]:
embeddingColumns = [str(i) for i in range(420)]
columnNames = ["START", "END"] + embeddingColumns
path = './ScreamingPihaEmbeddings/'

In [3]:
def generate_embeddings_from_file(path, filename):
    with open(path + filename, 'r') as f:
        data = f.read()
    with open(path + filename, 'w') as f:
        f.write(",".join(data.split("\t")))
    file_df = pd.read_csv(path + filename, names = columnNames)
    file_df["FILE"] = filename[:filename.index(".birdnet")]
    return file_df

def generate_embeddings(path):
    df = pd.DataFrame()
    for filename in os.listdir(path):
        df = pd.concat([df, generate_embeddings_from_file(path, filename)])
    df["PATH"] = path
    columns = df.columns.tolist()
    columns = columns[-2:] + columns[:-2]
    df = df[columns]
    df = df.reset_index()
    return df

In [4]:
embeddings_df = generate_embeddings(path)
embeddings_df

,index,FILE,PATH,START,END,0,1,2,3,4,...,410,411,412,413,414,415,416,417,418,419
0,0,ScreamingPiha5,./ScreamingPihaEmbeddings/,0.0,3.0,0.575971,0.528098,0.493685,0.656782,0.550628,...,0.591935,0.151428,0.610771,0.509292,1.380094,0.581467,0.785838,1.357152,0.342564,0.241052
1,1,ScreamingPiha5,./ScreamingPihaEmbeddings/,3.0,6.0,0.552567,0.154642,0.801572,0.467878,0.239668,...,0.511688,0.016979,0.843600,0.266170,0.462688,0.440184,0.125745,1.103997,0.345497,0.332996
2,2,ScreamingPiha5,./ScreamingPihaEmbeddings/,6.0,9.0,0.420057,0.835832,0.495816,0.454529,1.072033,...,0.869443,0.678777,1.148105,0.163387,1.338580,0.742934,0.342438,1.549533,0.647971,0.322725
3,3,ScreamingPiha5,./ScreamingPihaEmbeddings/,9.0,12.0,0.833276,0.498665,0.706026,1.221450,1.520700,...,0.544728,0.413031,1.322562,0.452439,1.025363,0.757578,0.248885,1.936727,1.057370,0.679428
4,4,ScreamingPiha5,./ScreamingPihaEmbeddings/,12.0,15.0,0.311551,0.191104,0.677158,0.181591,0.279591,...,0.428159,0.063017,0.335554,0.576328,0.435092,0.494634,0.071414,0.579821,0.323651,0.260343
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,1,ScreamingPiha3,./ScreamingPihaEmbeddings/,3.0,6.0,0.425072,0.490800,0.919007,0.640793,0.996168,...,0.558794,0.512046,0.825051,0.366001,0.801363,0.701596,0.149259,0.863319,0.972427,0.678520
184,0,ScreamingPiha4,./ScreamingPihaEmbeddings/,0.0,3.0,0.640207,0.305223,0.083695,0.030619,0.513983,...,0.939261,0.849727,0.384641,0.897279,0.593215,0.303327,0.998153,0.585512,0.456127,0.719388
185,1,ScreamingPiha4,./ScreamingPihaEmbeddings/,3.0,6.0,0.678870,0.346216,0.443378,0.197604,0.390667,...,0.342808,0.627806,0.885634,0.755843,0.844591,0.638393,0.428948,1.039371,0.583169,0.422725
186,2,ScreamingPiha4,./ScreamingPihaEmbeddings/,6.0,9.0,0.470342,0.363164,0.316475,0.192720,0.342391,...,0.303668,1.171068,0.442895,0.419631,0.243802,0.320295,0.349189,0.213859,0.377708,0.087124


### Centroid Exploration